# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.wavemeter import wlm_monitor
from pylabnet.network.client_server import nidaqmx_card
from pylabnet.utils.sweeper import Sweep1D

Enter host and port parameters here:

In [2]:
logger_host = '140.247.189.125'
logger_port = 6067
wavemeter_host = '140.247.189.125'
wavemeter_port = 1477
nidaqmx_host = '140.247.189.125'
nidaqmx_port = 3604

Instantiate all clients

In [3]:
log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
wm = wlm_monitor.Client(
    host=wavemeter_host,
    port=wavemeter_port,
)
ni = nidaqmx_card.Client(
    host=nidaqmx_host,
    port=nidaqmx_port
)
exp = Sweep1D(logger=log)

## Experiment

Parameters, setup

In [4]:
f0 = 406.693
f1 = 406.695
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts)
counter = ni.create_timed_counter(
    counter_channel='Dev1/Ctr0',
    physical_channel='/Dev1/PFI0'
)

Experiment script

In [5]:
def experiment(frequency):

    # Set frequency
    wm.update_parameters([{
        'channel': 1, 'setpoint': frequency
    }])

    # Activate counter
    ni.start_timed_counter(counter)
    time.sleep(0.1)

    # Get counts/sec
    return ni.get_count(counter)*10
    

In [6]:
exp.configure_experiment(experiment)

In [7]:
exp.run()

## Analyze data

In [9]:
exp.y_data

array([296610., 296610., 296620., 296610., 296620., 296610., 296610.,
       296580., 296620., 296610., 296610., 296620., 296580., 296620.,
       296590., 296600., 296600., 296620., 296610., 296610., 364190.,
       296600., 296590., 296610., 296600., 296610., 296610., 296620.,
       296600., 296590., 364180., 296620., 364210., 296600., 296610.,
       296620., 296620., 364200., 296590., 296600., 296620., 296590.,
       296600., 296590., 296620., 364190., 364190., 296590., 364200.,
       296610., 296600., 296600., 296600., 296620., 296590., 296620.,
       296590., 296610., 296600., 296600., 296580., 296600., 296610.,
       296610., 296610., 296600., 296590., 296610., 364220., 296620.,
       296610., 296610., 296600., 364190., 296600., 296600., 296600.,
       296600., 296620., 296590., 296590., 296610., 296600., 296590.,
       364210., 296590., 296600., 296610., 296610., 296590., 296590.,
       296600., 364210., 296620., 296630., 296590., 296580., 296590.,
       296590., 2966

## Close counter

In [5]:
ni.close_timed_counter('counter_0')